In [3]:
import os, uuid
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [4]:
!pip install azure-storage-blob

You should consider upgrading via the 'C:\Users\tempe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
# Bring necessary packages
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

Azure Devops 가면 key가 2개가 있는데

원래 작업중 코드를 짠다면, 첫번째 키 값이 실패할 경우를 대비해서 2번째 연결하는 코드또한 만들어 놓아야함!

일단 간단한 작업임으로, key1의 연결문자열을 통해 연결

In [7]:
# Connect to Blob Storage
connect_string = 'key1의 열결문자열'

blob_service_client = BlobServiceClient.from_connection_string(connect_string) # create Connected Blob Client

In [10]:
# Create New Container 
container_name = str(uuid.uuid4()) # 겹치지 않는 난수 문자열을 생성
print(container_name)

2d8371c1-a622-4ae0-920f-dfe2059991b5


In [11]:
container_client = blob_service_client.create_container(container_name) # 위의 난수 문자들을 활용해서 컨테이너 생성

In [12]:
# Upload the data to the container
local_path = './data'
os.mkdir(local_path) # create new folder in the directory

local_file_name = str(uuid.uuid4()) + '.txt'
upload_file_path = os.path.join(local_path, local_file_name) 
upload_file_path

'./data\\edbdcaba-c172-4f82-8566-a35318df36e5.txt'

In [13]:
# Prepare the data file to upload
file = open(file=upload_file_path, mode='w')
file.write('Hello Azure Storage!')
file.close()

In [14]:
# 새로운 blob 클라이언트 생성
blob_client = blob_service_client.get_blob_client( container = container_name, blob = local_file_name) 

In [16]:
# *** 실질적인 업로딩 코드 ***
# binary 단위로 텍스트를 읽어들여서 파일을 업로드
with open(file=upload_file_path, mode='rb') as data:
    blob_client.upload_blob(data)

ResourceExistsError: The specified blob already exists.
RequestId:4936d598-801e-002f-0cc5-04a557000000
Time:2022-11-30T14:12:20.1586923Z
ErrorCode:BlobAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobAlreadyExists</Code><Message>The specified blob already exists.
RequestId:4936d598-801e-002f-0cc5-04a557000000
Time:2022-11-30T14:12:20.1586923Z</Message></Error>

html파일을 업로딩 할 시, azure 서버를 통해서 웹사이트 제작 가능!!

파일을 업로딩 해준 뒤, 액세스 수준 변경을 통해서 private -> blob or container로 변경해줘야지 올린 파일 다운로드 가능

In [17]:
# Check file list from the client
blob_list = container_client.list_blobs()
for blob in blob_list:
    print('\t' + blob.name)

	edbdcaba-c172-4f82-8566-a35318df36e5.txt


In [20]:
# File Donwload from the uploaded Azure Storage Client
download_file_path  = os.path.join(local_path, str.replace(local_file_name, '.txt', 'DOWNLOAD.txt'))

container_client = blob_service_client.get_container_client(container=container_name)

# 파일이 날라올때 스트링으로 날라오기 때문에 
with open(file=download_file_path, mode='wb') as download_file:
    download_file.write(container_client.download_blob(blob.name).readall()) #어떤 파일을 다운받을지, 전체를 다운받으라고 세팅함

data 파일을 확인해보면 original txt파일과 DONWLOAD된 파일을 확인 해 볼 수 있음!

In [ ]:
# 실습한 자원의 정리
print('Pree the Enter key to begin clean up')
input()

print('Deleting blob container...')
container_client.delete_container()

print('Delting the local source and downloaded files...')
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)

print('Done')